# Small Product Mix Example

Your firm produces three products. Each product must go through three different departments before it is complete, taking various amounts of time for each step. Every product has an associated profit for each unit sold but also has a maximum demand for each month. Each  department has limited labor available each month. It also costs to set up the machines associated with each product. What is the best production plan for the next month?

## Data

| | | | |
|:--- | :----: | :----: | :----: |
| | **Product 1** | **Product 2** | **Product 3** |
| Profit per unit | \$25 | \$28 | \$30 |
| Dept A Labor Hours | 1.5 | 3 | 2 | 
| Dept B Labor Hours | 2 | 1 | 2.5 |
| Dept C Labor Hours | 0.25 | 0.25 | 0.25 |
| Maximum Production | 175 | 150 | 140 |

We have 450 hours in Dept A, 350 hours in Dept B, and 50 hours in Dept C.

Oh, by the way, it costs \\$400 to setup the machines for Product 1, \\$550 to setup the machines for Product 2, and \\$600 to setup the machines for Product 3.

How do you want to tackle this problem?

## Simplify

Let's first simply ignore the setup costs and simply see if we can get a production plan for the 3 products. Here is a variable-by-variable formulation.

| | | |
|---|---:|---|
| Let | $x_{1}$ = | number of units of product 1 to produce |
| | $x_{2}$ = | number of units of product 2 to produce |
| | $x_{3}$ = | number of units of product 3 to produce |

| | | | | | | | | |
|---|---:|---|---:|---|---:|---|---:|---|
|$\max$|$25x_{1}$|+|$28x_{2}$|+|$30x_{3}$| | |
|s.t.| |||||
||$1.5x_{1}$|+|$3x_{2}$|+|$2x_{3}$|$\leq$|$450$| {Dept A labor hours} |
||$2x_{1}$|+|$x_{2}$|+|$2.5x_{3}$|$\leq$|$350$| {Dept B labor hours} |
||$0.25x_{1}$|+|$0.25x_{2}$|+|$0.25x_{3}$|$\leq$|$50$| {Dept C labor hours} |
|$0\leq$|$x_{1}$| | | | |$\leq$ | $175$| {bounds for $x_{1}$} |
|$0\leq$| | |$x_{2}$ | | |$\leq$ | $150$| {bounds for $x_{2}$} |
|$0\leq$| | | | |$x_{3}$ |$\leq$ | $140$| {bounds for $x_{3}$} |

In [ ]:
# Import the packages/modules needed
import gurobipy as gp
from gurobipy import GRB
import sensitivity_analysis as sa

In [ ]:
# Set up the data for the problem
''' Data in Python Lists '''
cnstr_coeff = [[1.5, 3.0, 2.0],
               [2.0, 1.0, 2.5],
               [0.25, 0.25, 0.25]]
cnstr_names = ['DeptA_hrs', 'DeptB_hrs', 'DeptC_hrs']
cnstr_rhs = [450.0, 350.0, 50]
obj_coeff = [25, 28, 30]
var_names = ['prod_1', 'prod_2', 'prod_3']
ub = [175, 150, 140]

In [ ]:
# Create the Gurobi model
m = gp.Model('small_prod_mix')

# Specify how to optimize
m.ModelSense = GRB.MAXIMIZE

# Create the decision variables and put in a list
# Create empty list
dvars = []

# Use a standard for loop to append each variable into list
for i in range(len(var_names)):
    dvars.append(m.addVar(vtype=GRB.CONTINUOUS,
                          name=var_names[i],
                          lb=0.0,
                          ub=ub[i]))

m.update()
m.display()

In [ ]:
# Create the objective function
m.setObjective(gp.quicksum(obj_coeff[i]*dvars[i] for i in range(len(dvars))))
m.update()
m.display()

In [ ]:
# Create the constraints using a for loop
for i in range(len(cnstr_coeff)):
    m.addLConstr(gp.quicksum((cnstr_coeff[i][j]*dvars[j] for j in range(len(dvars)))),
                GRB.LESS_EQUAL,
                rhs=cnstr_rhs[i],
                name=cnstr_names[i])

m.update()
m.display()

In [ ]:
# Optimize the model
m.optimize()

In [ ]:
# Get the results
print(f'Optimal objective function: ${m.ObjVal:0.2f}')
for v in m.getVars():
    print(f'  {v.varName} = {v.X}')

In [ ]:
# Look at sensitivity information
sa.sa_vars(m.getVars())

In [ ]:
sa.sa_constrs(m.getConstrs())

## But What about Fixed Costs

We ignored the fixed costs for setting up the machines. In this answer, we produced all three products. That means we need to set up the machines for each product incurring additional costs of \\$400 for Product 1, \\$550 for Product 2, and \\$600 for Product 3 for a total of \\$1550. Subtracting these fixed setup costs from our objective function value of \\$5,540 gives a "true" profit of \\$3,390.

# Incorporating Fixed Costs Correctly

It should seem reasonable that solving the problem as we just did may not be the best option. Instead, we should take the fixed costs of setting up the machines for each product into account from the beginning. Doing so will allow us to determine if we should **not** produce all three products because the fixed costs may outweigh the profit margins for a particular product.

We can incorporate fixed costs by introducing **binary** variables where a 1 will indicate that we decide to produce the particular product and 0 will indicate that we do **not** produce the particular product. It follows, then, that we will need a binary variable for each of the products. Therefore we need to introduce three binary variables.

Where will this binary variables show up? Because it costs to set up the machines for an associated product, it should seem obvious that the binary variable will show up in the objective function. The coefficient associated with each binary variable will be the corresponding setup cost for that product. That's the straight-forward part.

We now need to take care of how many of each product to produce. We know from before that we have upper bounds on each product. Now, we need to make sure that we only produce a product if we have decided to set up the machines for it. That is, if the binary variable is "on", i.e. a 1, then we can produce up to the limit as before. If, however, the binary variable is "off", i.e. a 0, then we cannot produce any of that product because the machines have not been set up to for a production run of that product. 

## The New Formulation

| | | |
|---|---:|---|
| Let | $x_{1}$ = | number of units of product 1 to produce |
| | $x_{2}$ = | number of units of product 2 to produce |
| | $x_{3}$ = | number of units of product 3 to produce |
| | $y_{1}$ = | $\begin{cases}
      1 & \text{if setup machines to produce product $1$}\\
      0 & \text{otherwise}
    \end{cases}$ |
| | $y_{2}$ = | $\begin{cases}
      1 & \text{if setup machines to produce product $2$}\\
      0 & \text{otherwise}
    \end{cases}$ |
| | $y_{3}$ = | $\begin{cases}
      1 & \text{if setup machines to produce product $3$}\\
      0 & \text{otherwise}
    \end{cases}$ |

| | | | | | | | | | | | | | | |
|---|---:|---|---:|---|---:|---|---:|---|---|---|---|---|---|---|
|$\max$|$25x_{1}$|+|$28x_{2}$|+|$30x_{3}$|+|$400y_{1}$|+|$550y_{2}$|+|$600y_{3}$ | | | |
|s.t.| ||||||||||||||
||$1.5x_{1}$|+|$3x_{2}$|+|$2x_{3}$|||||||$\leq$|$450$| {Dept A labor hours} |
||$2x_{1}$|+|$x_{2}$|+|$2.5x_{3}$|||||||$\leq$|$350$| {Dept B labor hours} |
||$0.25x_{1}$|+|$0.25x_{2}$|+|$0.25x_{3}$|||||||$\leq$|$50$| {Dept C labor hours} |
||$x_{1}$| | | | |$-$|$175y_{1}$|||||$\leq$ |$0$ | {maximum production for product 1} |
|| | |$x_{2}$ | | |||$-$| $150y_{2}$|||$\leq$ |$0$| {maximum production for product 2} |
|| | | | |$x_{3}$ |||||$-$| $140y_{3}$|$\leq$ |$0$| {maximum production for product 3} |
||$x_{1}$|,|$x_{2}$|,|$x_{3}$|||||||$\geq$| $0$ | {non-negativity for producing products} |

In [ ]:
# Create the model
prod_mix = gp.Model('product_mix')
prod_mix.ModelSense = GRB.MAXIMIZE

In [ ]:
# Create setup costs
setup = [400, 550, 600]

In [ ]:
# Create the variables


In [ ]:
# Set up the objective function


In [ ]:
# Add constraints
# Department labor hours


# Maximum production based on setting up machines


In [ ]:
# Optimize
prod_mix.optimize()

In [ ]:
# Get the results
print(f'Optimal objective function: ${prod_mix.ObjVal:0.2f}')
for v in prod_mix.getVars():
    print(f'  {v.varName} = {v.X}')